In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
def get_info(soup):
    nameList = soup.find_all('span', class_='courseNumber')
    k = [name.text.strip()[:-1] for name in nameList]

    units = soup.find_all('div', class_='courseAttributes')
    v = []
    for unit in units:
        if unit:
            text = unit.find(string=lambda text: "Units:" in text)
            if text:
                s = text.split("Units:")[1].strip()[:3]
                v.append(s)
    unit_map = {course: unit for course, unit in zip(k, v)}
    return helper(unit_map, soup)

def helper(unit_map, soup):
    details = soup.find_all('li', class_='sectionDetails')
    all_sections = []
    for detail in details:
        section = []
        text = detail.get_text(strip=True)
        name = text.split("|")[0].strip()
        section.append(name)
        section.append(unit_map.get(name))
        brTags = detail.find_all('br')
        for brTag in brTags:
            s = brTag.next_sibling.strip()
            if (len(s) > 0 and 'Exam Date' not in s):
                temp = section.copy()
                temp.append(s)
                all_sections.append(temp)
    return all_sections

def sub_page(soup):
    div_tag = soup.find('div', id="pagination")
    a_tags = div_tag.find_all('a')
    href_values = [a['href'] for a in a_tags][:-1]
    return [url + h for h in href_values]

def all_url(soup):
    li_tags = soup.find_all('li')
    result = []
    for li in li_tags:
        a_tags = li.find_all('a')
        for a in a_tags:
            result.append(url + a['href'])
    return result

In [3]:
url = 'https://explorecourses.stanford.edu/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")

In [4]:
# sample output for CS class
# url_list contains all url from main page in all major
url_list = all_url(soup)
dep_url = url_list[32]
result_list = []
dep_soup = BeautifulSoup(requests.get(dep_url, headers=headers).content, "html.parser")
result_list.extend(get_info(dep_soup))
pages = sub_page(dep_soup)
for page in pages:
    sub_soup = BeautifulSoup(requests.get(page, headers=headers).content, "html.parser")
    result_list.extend(get_info(sub_soup))

for i in result_list[:10]:
    print("============================")
    print('Name:', i[0])
    print('Units:', i[1])
    print('Time:', i[2])

Name: CS 12SI
Units: 1
Time: 04/01/2024 - 06/05/2024 Mon, Wed 4:30 PM - 5:20 PM at
Name: CS 21SI
Units: 2
Time: 04/01/2024 - 06/05/2024 Wed 5:30 PM - 7:20 PM at
Name: CS 52
Units: 2
Time: 04/01/2024 - 06/05/2024 Tue, Thu 4:30 PM - 5:50 PM at
Name: CS 53N
Units: 3
Time: 04/01/2024 - 06/05/2024 Tue, Thu 10:30 AM - 11:50 AM at
Name: CS 100ACE
Units: 1
Time: 09/26/2023 - 12/08/2023 Wed 3:30 PM - 5:20 PM at
Name: CS 100ACE
Units: 1
Time: 01/08/2024 - 03/15/2024 Wed 3:30 PM - 5:20 PM at
Name: CS 100ACE
Units: 1
Time: 04/01/2024 - 06/05/2024   -
Name: CS 100BACE
Units: 1
Time: 09/26/2023 - 12/08/2023 Mon 4:30 PM - 6:20 PM at
Name: CS 100BACE
Units: 1
Time: 01/08/2024 - 03/15/2024 Tue 1:30 PM - 3:20 PM at
Name: CS 100BACE
Units: 1
Time: 04/01/2024 - 06/05/2024   -


In [ ]:
url_list = all_url(soup)
for dep_url in url_list:
    dep_url = url_list[3]
    result_list = []
    dep_soup = BeautifulSoup(requests.get(dep_url, headers=headers).content, "html.parser")
    result_list.extend(get_info(dep_soup))
    pages = sub_page(dep_soup)
    for page in pages:
        sub_soup = BeautifulSoup(requests.get(page, headers=headers).content, "html.parser")
        result_list.extend(get_info(sub_soup))